In [6]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Step 1: Data Preparation
data_dir = 'img'
image_size = (32, 32)  # Adjust the size as needed

def load_data():
    images = []
    labels = []
    for label, folder in enumerate(sorted(os.listdir(data_dir))):
        for img_file in os.listdir(os.path.join(data_dir, folder)):
            img_path = os.path.join(data_dir, folder, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, image_size) / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_data()
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Step 2: Model Definition
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(26, activation='softmax')  # 26 output classes for the alphabet
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 3: Training
model.fit(X_train.reshape(-1, image_size[0], image_size[1], 1), y_train, epochs=100, batch_size=32, validation_split=0.2)

# Step 4: Evaluation
test_loss, test_acc = model.evaluate(X_test.reshape(-1, image_size[0], image_size[1], 1), y_test)
print("Test Accuracy:", test_acc)


Epoch 1/100


C:\Users\ASUS\anaconda3\envs\Open_cv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.0430 - loss: 3.2624 - val_accuracy: 0.0906 - val_loss: 3.2366
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1245 - loss: 3.1776 - val_accuracy: 0.1772 - val_loss: 2.7976
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3040 - loss: 2.4692 - val_accuracy: 0.4961 - val_loss: 2.0572
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4920 - loss: 1.8385 - val_accuracy: 0.5748 - val_loss: 1.6691
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6086 - loss: 1.4724 - val_accuracy: 0.6142 - val_loss: 1.4405
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6421 - loss: 1.2825 - val_accuracy: 0.6496 - val_loss: 1.3544
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6749 - loss: 1.1202 - val_accuracy: 0.6614 - val_loss: 1.2216
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7286 - loss: 0.9871 - val_accuracy: 0.6732 - val_l

In [7]:
# Save the trained model
model.save("braille_model.h5")


In [13]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("braille_model.h5")

def preprocess_image(image_path):
    # Load and preprocess the image
    if not os.path.isfile(image_path):
        raise ValueError("Invalid image file path. File does not exist.")
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Invalid image file format.")
    img = cv2.resize(img, (32, 32)) / 255.0  # Resize and normalize
    return img.reshape(-1, 32, 32, 1)

def classify_image(image_path):
    try:
        # Preprocess the image
        img = preprocess_image(image_path)
        # Perform classification
        predictions = model.predict(img)
        # Get the predicted class
        predicted_class = chr(ord('a') + np.argmax(predictions))
        return predicted_class
    except Exception as e:
        raise ValueError("Error classifying the image:", e)

def main():
    # Ask the user to input an image path
    image_path = input("Enter the path to the image you want to classify: ")
    try:
        # Classify the image
        predicted_class = classify_image(image_path)
        print("Predicted letter:", predicted_class)
    except ValueError as ve:
        print(ve)

if __name__ == "__main__":
    main()


Enter the path to the image you want to classify:  12.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Predicted letter: l
